In [ ]:
import numpy as np
from urllib.request import urlopen
import json
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
%matplotlib inline

In [ ]:
url = "https://api.rated.network/v0/eth/operators?window=7d&idType=depositAddress&from=10000&size=1000"

In [ ]:
response = urlopen(url)

In [ ]:
data_json = json.loads(response.read())

In [ ]:
print(len(data_json['data']))

In [ ]:
validatorCounts = []

In [ ]:
for i in range(1000):
    validatorCounts.append(data_json['data'][i:i+1][0].get('validatorCount'))
    #print(data_json['data'][i:i+1][0].get('id'),data_json['data'][i:i+1][0].get('validatorCount'))

In [ ]:
plt.hist(validatorCounts, bins=100, density=False)
plt.yscale('log')
#plt.xscale('log')
plt.xlabel('Stake size (in ETH)')
plt.ylabel('Count')
plt.title('Stake distribution of the Ethereum Proof-of-stake validators')
plt.savefig('figures/stakeDistribution.pdf')
plt.show()

In [ ]:
len(validatorCounts)
np.array(validatorCounts).dump(open('figures/validatorCounts.npy', 'wb'))

In [ ]:
sendingProbabilities = []
for i in range(len(validatorCounts)):
    prob = validatorCounts[i]/sum(validatorCounts)
    sendingProbabilities.append(prob)
np.array(validatorCounts).dump(open('figures/sendingProbabilities.npy', 'wb'))

In [ ]:
pdf = np.load(open('figures/sendingProbabilities.npy', 'rb'), allow_pickle=True)

In [ ]:
count, bins_count = np.histogram(validatorCounts, bins=100)

In [ ]:
len(pdf)
print(pdf[:20])

In [ ]:
cdf = np.cumsum(pdf)
cdf = cdf/np.sum(pdf)
pdf = pdf/np.sum(pdf)
print(pdf[:10])

In [ ]:
# plotting PDF and CDF
#plt.plot(pdf, color="red", label="PDF")
plt.plot(cdf, label="CDF")
plt.plot(pdf, label="PDF")
plt.xscale('log')
plt.legend()
plt.xlabel('Validator count')
plt.ylabel('Probability')
plt.title('PDF and CDF of sending probabilities')
plt.savefig('figures/pdfVScdfSendingProbabilities.pdf')
plt.show()

In [ ]:
G=nx.Graph()

In [ ]:
file1 = open('figures/goerliTestnetTopology.txt', 'r')
Lines = file1.readlines()
  
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    myLine = line.strip()
    firstNode = int(myLine[4:4+myLine[4:].find(' ')])
    secondNode = int(myLine[4+myLine[4:].find(' ')+12:])
    if firstNode not in G.nodes:
        G.add_node(firstNode)
    if secondNode not in G.nodes:
        G.add_node(secondNode)
    G.add_edge(firstNode, secondNode)

In [ ]:
G.number_of_nodes(), G.number_of_edges()

In [ ]:
scores = list(dict(G.degree()).values())
plt.hist(scores, bins=50)
plt.title("Degree")
plt.show()

In [ ]:
scores = list(dict(nx.pagerank(G)).values())
plt.hist(scores, bins=50)
plt.title("PageRank")
plt.show()

In [ ]:
nx.write_edgelist(G, "figures/goerliTopology.edgelist", data=False)

In [ ]:
nx.write_gpickle(G, "figures/goerliTopology.gpickle")